<img src="python_ecosystem.png">

(image: Ondřej Čertík)

In [1]:
import numpy as np

In [2]:
lst = list(range(10))

a = np.asarray(lst)

a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Arithmetic operations with arrays typically work elementwise

In [3]:
a + 1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [4]:
a * 2

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [5]:
lst * 2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
a**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

####  Boolean arrays

In [7]:
a**2 < 42

array([ True,  True,  True,  True,  True,  True,  True, False, False, False], dtype=bool)

In [8]:
lst < 42

False

In [9]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
a[a**2 < 42]

array([0, 1, 2, 3, 4, 5, 6])

In [11]:
10 < a**2 < 42

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [14]:
(10 < a**2) & (a**2 < 42)

array([False, False, False, False,  True,  True,  True, False, False, False], dtype=bool)

Note the need to use the bitwise-AND operator and brackets.

**Caveat**: Assignments with boolean indexing modify the array in-place

In [15]:
a[(10 < a**2) & (a**2 < 42)] = -42

In [16]:
a

array([  0,   1,   2,   3, -42, -42, -42,   7,   8,   9])

More info: search for *fancy indexing* vs *basic indexing*

#### Arrays have useful methods

In [17]:
len(dir(a))

163

In [18]:
am = (a - a.mean()) / a.std()

In [19]:
# compare to
from __future__ import division

a_mean = sum(x for x in a) / len(a)

from math import sqrt
a_std = sqrt(sum((x - a_mean)**2 for x in a) / len(a))

am_list = [(x - a_mean) / a_std for x in a]

In [20]:
am_list - am

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [21]:
am.median()

AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [22]:
np.median(a)

1.5

#### Arrays can be reshaped in `O(1)` in time and memory

In [23]:
a.shape

(10,)

In [24]:
a.reshape(2, -1)

array([[  0,   1,   2,   3, -42],
       [-42, -42,   7,   8,   9]])

In [25]:
b = a.reshape(-1, 2)

In [26]:
b

array([[  0,   1],
       [  2,   3],
       [-42, -42],
       [-42,   7],
       [  8,   9]])

In [27]:
a

array([  0,   1,   2,   3, -42, -42, -42,   7,   8,   9])

#### Elementwise operations can be applied along an axis

In [28]:
np.mean(b, axis=1)

array([  0.5,   2.5, -42. , -17.5,   8.5])

In [29]:
np.mean(b, axis=1, keepdims=True)

array([[  0.5],
       [  2.5],
       [-42. ],
       [-17.5],
       [  8.5]])

#### Slicing

Neighborhood average of a two-dim array

> A general issue of speed for the overall program. A single run with sufficient data points is taking about 2-3 weeks.


In [30]:
m, n = 4, 4
a = np.arange(m*n, dtype=np.float).reshape((m, n))
a

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.],
       [ 12.,  13.,  14.,  15.]])

In [31]:
# a non-vectorized way

b = np.zeros((m-1, n-1))
for i in range(m-1):
    for j in range(n-1):
        b[i, j] = a[i, j] + a[i+1, j] + a[i, j+1] + a[i+1, j+1]
b

array([[ 10.,  14.,  18.],
       [ 26.,  30.,  34.],
       [ 42.,  46.,  50.]])

In [32]:
# the syntax for a slice is `start:stop:step`

a[1:3, 0]

array([ 4.,  8.])

In [33]:
a[1:-1, ...]

array([[  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [34]:
a[1:, ...]

array([[  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.],
       [ 12.,  13.,  14.,  15.]])

In [35]:
b_vect = a[:-1, :-1] + a[1:, :-1] + a[:-1, 1:] + a[1:, 1:]

In [36]:
np.all(b_vect == b)

True

In [37]:
N = 1000
np.random.seed(1234)
r = np.random.random((N, N))

In [38]:
%%timeit 

r_av = np.zeros((N-1, N-1))
for i in range(N-1):
    for j in range(N-1):
        r_av[i, j] = r[i, j] + r[i+1, j] + r[i, j+1] + r[i+1, j+1]

1 loops, best of 3: 1.13 s per loop


In [39]:
%%timeit 

r_av = r[:-1, :-1] + r[1:, :-1] + r[:-1, 1:] + r[1:, 1:]

100 loops, best of 3: 7.94 ms per loop


#### Conway's game of life

Cells live on a square grid. Each cell can be in either of two states: alive or dead. Cells interact with nearest neighbors.

At each *tick*, 

- Any live cell with `<2` neighbors dies, as if of underpopulation.
- Any live cell with `>3` neighbors dies, as if of overpopulation.
- Any dead cell with `=3` neighbors becomes a live cell, as if by reproduction.

From a cell-centric view to a whole-array formulation: for each cell, consider the sum of nine fields.
- If the sum `= 3`, central cenral cell's state is life.
- If the sum `= 4`, the state of the central cell does not change
- Otherwise, it dies.

In [40]:
def step(X):
    """Given a game board ``X``, make a time step and return the result.
    
    NB: In this implementation the game field is finite.

    """
    num_neighb = (X[:-2, :-2]  + X[1:-1, :-2]  + X[2:, :-2] +
                  X[:-2, 1:-1] + X[1:-1, 1:-1] + X[2:, 1:-1] +
                  X[:-2, 2:]   + X[1:-1, 2:]   + X[2:, 2:])
    
    X[1:-1, 1:-1][num_neighb == 3] = 1
    X[1:-1, 1:-1][(num_neighb != 4) & (num_neighb != 3)] = 0
    return X

https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life

In [41]:
from scipy.signal import convolve2d

window = np.ones((3, 3))

def step_alternative(X):
    nbrs_count = convolve2d(X, window, mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

### Broadcasting

OP:

> I personally think that silent Broadcasting is not a good thing. I had recently a lot of trouble with row and column vectors which got bradcastet toghether ...

Chuck Harris:

> It's how numpy works. 

In [42]:
a = np.arange(5)
a

array([0, 1, 2, 3, 4])

In [43]:
a[:, None]

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [44]:
a[:, None].shape

(5, 1)

In [45]:
a + a[:, None]

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

#### `numpy` gotchas for python users

`lst[:]` is a copy of `lst`

`arr[:]` is a *view* into `arr` (for copying use `arr.copy()`)

#### `numpy` gotchas for pandas users

In [46]:
a = np.array([np.nan, 2., 3.])

import pandas as pd
s = pd.Series(a)

In [47]:
s.sum()

5.0

In [48]:
a.sum()

nan

In [49]:
np.nansum(a)

5.0

In `numpy`, NaN means "invalid" not "missing".

## Universal functions, `ufuncs`

In [50]:
a = np.array(list(range(10)), dtype=np.float)

In [51]:
np.sin(a)

array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])

In [52]:
am = (a - a.mean()) / a.std()

1 / (1. + np.exp(am))

array([ 0.8273125 ,  0.77180715,  0.70482648,  0.62766976,  0.54340985,
        0.45659015,  0.37233024,  0.29517352,  0.22819285,  0.1726875 ])

In [53]:
np.add(a, a)

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.])

In [54]:
np.multiply(a, a)

array([  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81.])

**NB**: `np.multiply` is *elementwise* multiplication. For linear algebra operations, use `np.dot`

### In-place operations

In [68]:
a = np.array(list(range(10)), dtype=np.float)
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [60]:
opts = np.set_printoptions(precision=3)

In [66]:
np.exp(a, out=a)

array([  1.000e+00,   2.718e+00,   7.389e+00,   2.009e+01,   5.460e+01,
         1.484e+02,   4.034e+02,   1.097e+03,   2.981e+03,   8.103e+03])

In [67]:
a

array([  1.000e+00,   2.718e+00,   7.389e+00,   2.009e+01,   5.460e+01,
         1.484e+02,   4.034e+02,   1.097e+03,   2.981e+03,   8.103e+03])

#### `np.add.<TAB>`

- `np.add.reduce` is `np.sum`

- `np.add.accumulate` is `np.cumsum`

- `np.add.outer` has the outer-product semantics

#### Cauchy matrix

Given two arrays $u_i$ and $v_i$, construct

$$
A_{ij} = \frac{1}{u_i - v_j}
$$

In [55]:
u = np.arange(3)
v = np.arange(3) + 0.5

A_ = np.zeros((len(u), len(v)))
for i in range(len(u)):
    for j in range(len(v)):
        A_[i, j] = 1. / (u[i] - v[j])
A_

array([[-2.        , -0.66666667, -0.4       ],
       [ 2.        , -2.        , -0.66666667],
       [ 0.66666667,  2.        , -2.        ]])

In [56]:
A = 1. / np.subtract.outer(u, v)
A

array([[-2.        , -0.66666667, -0.4       ],
       [ 2.        , -2.        , -0.66666667],
       [ 0.66666667,  2.        , -2.        ]])

### There's lots of resourses online

Numpy docs http://docs.scipy.org/doc/numpy/reference/index.html

Jake Vanderplas' *Numpy Intro*
http://nbviewer.ipython.org/github/jakevdp/2013_fall_ASTR599/blob/master/notebooks/05_NumpyIntro.ipynb

and *Efficient Numpy* http://nbviewer.ipython.org/github/jakevdp/2013_fall_ASTR599/blob/master/notebooks/11_EfficientNumpy.ipynb

Scipy lecture notes, incl Pauli Virtanen's *Advanced Numpy*
https://scipy-lectures.github.io/

Nicolas Rougier's *100 Numpy exercises*
http://www.labri.fr/perso/nrougier/teaching/numpy.100/